<a href="https://colab.research.google.com/github/cstecker/comparative_politics/blob/main/Wie_die_Thesenauswahl_den_Wahl_O_Mat_beeinflusst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Wie die Thesenauswahl den Wahl-O-Mat beeinflusst**

## **Setup**
Zu Beginn laden wir die Pakete, die wir benötigen.

In [9]:
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Sodann laden wir die Daten für den Wahl-O-Mat von Hessen, den wir nutzen, um ein paar Überlegungen zu illustrieren

In [7]:
# Datensatz herunterladen
download.file("https://github.com/cstecker/politicsRLab/blob/main/data/he2023.rds?raw=true", "he2023.rds")

# Datensatz in R einbinden
he2023 <- readRDS("he2023.rds")

**Der Wahl-O-Mat für Hessen 2023**



In [10]:
glimpse(he2023)

Rows: 798
Columns: 8
$ party_no      <dbl> 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 1…
$ party_short   <chr> "CDU", "GRÜNE", "SPD", "AfD", "FDP", "DIE LINKE", "FREIE…
$ party_long    <chr> "Christlich Demokratische Union Deutschlands", "BÜNDNIS …
$ these_id      <dbl> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,…
$ these_title   <chr> "Wählen mit 16", "Wählen mit 16", "Wählen mit 16", "Wähl…
$ these_content <chr> "Jugendliche sollen bei Landtagswahlen ab 16 Jahren wähl…
$ party_pos     <dbl> -1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, -1, 1, -1, 1…
$ party_reasons <chr> "Wir fördern eine stärkere Beteiligung junger Menschen a…


Schließlich können wir auch die oft vertraute "Excel-Perspektive" konsultieren (dazu wählen wir uns mal nur die SPD aus):

In [ ]:
he2023 %>%
    filter(party_short == "SPD") %>%
    View()

# Ihre eigenen Positionen zum Wahl-O-Mat in Hessen 2023

Nun geben Sie ihre Positionen zu den einzelnen Thesen ein.

In [13]:
# Creating the tibble with an self_pos column
self_pos <- tribble(
  ~`these_id`, ~`these_title`,                               ~self_pos,
  1,            "Wählen mit 16",                                "n",
  2,            "Abzug der US-Streitkräfte",                    "n",
  3,            "Schulessen",                                   "j",
  4,            "Nachtflugverbot",                              "n",
  5,            "Kohlekraftwerke",                              "j",
  6,            "Abschiebung",                                  "j",
  7,            "Ökologische Landwirtschaft",                   "n",
  8,            "Verfassungsschutz abschaffen",                 "n",
  9,            "Schuldenbremse",                               "j",
  10,           "Verpflichtendes letztes Kita-Jahr",            "j",
  11,           "Landärzte",                                    "j",
  12,           "Bürgergeld",                                   "j",
  13,           "Schiene vor Straße",                           "e",
  14,           "Ausbildungsplatzabgabe",                       "n",
  15,           "Wolf",                                         "j",
  16,           "Geschlechterneutrale Sprache",                 "n",
  17,           "Fünfprozenthürde",                             "j",
  18,           "Sozialwohnungen für Deutsche",                 "j",
  19,           "Bürgerwindparks",                              "j",
  20,           "Mehrgliedriges Schulsystem",                   "j",
  21,           "Hasskriminalität",                             "e",
  22,           "Landespflegegeld",                             "e",
  23,           "Flächenversiegelung",                          "j",
  24,           "Militärische Forschung",                       "j",
  25,           "Familienbild",                                 "j",
  26,           "Grundsteuer",                                  "e",
  27,           "Projekte gegen Rechtsextremismus",             "e",
  28,           "Frauenquote",                                  "n",
  29,           "Abgabe bei Nutzung von Grundwasser",           "e",
  30,           "Zwangsräumungen",                              "j",
  31,           "Paritätische Landeslisten",                    "n",
  32,           "Deutschtest",                                  "j",
  33,           "Strafrechtliche Verfolgung von Sitzblockaden", "j",
  34,           "Volksentscheide",                              "j",
  35,           "Tanzverbot",                                   "j",
  36,           "Entgeltfreier ÖPNV",                           "n",
  37,           "NSU-Akten",                                    "n",
  38,           "Verkaufsoffene Sonntage",                      "j"
) %>%
  # convert positions to numerical scale
  mutate(self_pos = case_when(
    self_pos == "n" ~ -1,
    self_pos == "e" ~ 0,
    self_pos == "j" ~ 1)
    )

In [14]:
comparison_data <- self_pos %>% select(these_id, self_pos) %>%
  left_join(he2023, by = "these_id") %>%
  filter(party_short %in% c("CDU", "GRÜNE", "SPD", "AfD", "FDP", "DIE LINKE", "FREIE WÄHLER"))


In [15]:
# calculate max possible distance
n_theses <- comparison_data %>%  distinct(these_id) %>% count()

# Calculate city block distance
cityblock_scores <- comparison_data %>%
  group_by(party_short) %>%
  # calcualte distance in each issue
  mutate(distance = abs(self_pos - party_pos))

cityblock_scores %>%
  group_by(party_short) %>%
  # calculate total distance and closeness
  mutate(
    total_distance = sum(distance),
    close_perc = 1 - (total_distance / (n_theses*2))) %>%
  distinct(party_short, total_distance, close_perc) %>%
  arrange(desc(close_perc))

  party_short  total_distance close_perc$n
1 CDU          15             0.8026316   
2 AfD          18             0.7631579   
3 FREIE WÄHLER 22             0.7105263   
4 FDP          26             0.6578947   
5 GRÜNE        35             0.5394737   
6 SPD          39             0.4868421   
7 DIE LINKE    59             0.2236842   

In [16]:
# add other matching algorithms and put it all in one tibble
similarity_scores <- comparison_data %>%
  group_by(party_short) %>%
  mutate(
    cityblock_distance = abs(self_pos - party_pos),
    euclidean_dist = (self_pos - party_pos)^2,
    scalar_product = self_pos * party_pos
  ) %>%
  summarise(
    total_distance = sum(cityblock_distance),
    close_perc = 1 - (total_distance / (n_theses * 2)),
    total_euclidean = sqrt(sum(euclidean_dist)),
    euclidean_perc = 1 - (total_euclidean / sqrt(n_theses * 2^2)),
    sum_scalar_product = sum(scalar_product),
    scalar_perc = sum_scalar_product / n_theses
  ) %>%
  mutate(
    hybrid_score = (close_perc + euclidean_perc + scalar_perc) / 3
  ) %>%
  arrange(desc(hybrid_score))

similarity_scores

  party_short  total_distance close_perc$n total_euclidean euclidean_perc$n
1 CDU          15             0.8026316     4.795832       0.6110067       
2 AfD          18             0.7631579     5.291503       0.5708025       
3 FREIE WÄHLER 22             0.7105263     5.830952       0.5270473       
4 FDP          26             0.6578947     6.782330       0.4498804       
5 GRÜNE        35             0.5394737     7.681146       0.3769768       
6 SPD          39             0.4868421     8.185353       0.3360802       
7 DIE LINKE    59             0.2236842    10.535654       0.1454456       
  sum_scalar_product scalar_perc$n hybrid_score$n
1  23                 0.60526316    0.67296714   
2  20                 0.52631579    0.62009205   
3  14                 0.36842105    0.53533157   
4  12                 0.31578947    0.47452154   
5   3                 0.07894737    0.33179929   
6  -1                -0.02631579    0.26553551   
7 -22                -0.57894737   -0.0699